In [0]:
import urllib
import pandas as pd
from bs4 import BeautifulSoup
import requests
import time
from random import randint
import codecs
import csv
import unicodedata
import re

In [0]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:
with open("/content/drive/My Drive/wiki_dataset/Links/movies1.html", "r") as f:
    contents = f.read()
    soup = BeautifulSoup(contents, 'lxml')
    
movies_soup= soup.find_all('tr')[1:]
dic = {}
for m in movies_soup:
    dic[int(m.find('td').text)]= m.find('a').text

movies_links = pd.DataFrame(list(dic.items()), columns= ['Id', 'Url'])

In [0]:
def crawl_wiki(movies):
    missing=[]
    for index ,m in movies.iterrows():
        try:
            id = m['Id']
            url = m['Url']

            html =requests.get(url)
            with open("article_"+str(id)+".html", "w") as file:
                file.write(html.text)
            time.sleep(randint(1, 3))
        except requests.exceptions.RequestException as e:
            print('Rate limited. Need to wait for 20 minutes')
            print(e)
            print('missing: ',index)
            missing.append(index)
            time.sleep(20*60)
    if len(missing)>0: crawl_wiki(movies.loc[missing])
            
        

In [0]:
def get_title(page):
    return page.find('h1', class_='firstHeading').get_text('i')

In [0]:
def get_intro(page):
    if page.find("div", class_="toc") != None:
        #print(1)
        content = page.find("div", class_="toc") #.findPreviousSiblings()
        pre = content.findPreviousSiblings()
        #pre.reverse()
        intro = ''
        for i in pre:
            if i.name =='p':
                intro+=i.text
            else: break 
        return intro
    elif page.find("table", class_="infobox vevent")!=None:
        #print(2)
        table = page.find("table", class_="infobox vevent")
        intro = ''
        for i in table.find_next_siblings():
            if i.name !='div':
                intro+=i.text

            else: break
        return intro
    else: return 'NA'

In [0]:
def get_plot(page):
    #pattern='(?:^|\W)Plot(?:$|\W)'
    plot_head= page.find(class_="mw-headline").find_parent()
    plot= ''
    for i in plot_head.find_next_siblings():
        if i.name != 'h2':
            plot+=i.text
        else: break
    return plot

In [0]:
def extract_infobox2(page):
    d = {'name':'NA', 'Directed by': 'NA', 'Produced by': 'NA','Written by': 'NA', 'Starring': 'NA', 'Music by': 'NA',
      'Release date': 'NA', 'Running time': 'NA', 'Country': 'NA','Language': 'NA', 'Budget': 'NA'}
    if page.find("table", class_="infobox vevent")==None: return list(d.values())
    table = page.find("table", class_="infobox vevent") #table infobox
    
    d['name'] = table.find("th", class_="summary").text if table.find("th", class_="summary") else 'NA'
    for record in table.findAll('tr'):
        for attr in record.findAll('th'):
            for data in record.findAll('td'):
                if attr.text in d:
                    if attr.text == 'Release date': #continue
                        d[attr.text] =unicodedata.normalize("NFKD",data.get_text(strip=True))
                    else:
                        d[attr.text] = ' - '.join(list(data.stripped_strings))
                else: continue 
    return list(d.values())

In [0]:
def create_tsv(p, r,f_name):
    missing_pages=[]
    with open(f_name, 'wt') as out_file:
        header=['Id', 'Title', 'Intro', 'Plot', 'Name', 'Directed_by', 'Produced_by','Written_by', 'Starring', 'Music_by',
        'Release_date', 'Running_time', 'Country','Language', 'Budget']
        tsv_writer = csv.writer(out_file, delimiter='\t')
        tsv_writer.writerow(header)
        for i in r:
            if i%1000==0: 
                print(i)
                time.sleep(60)
            #print(i)
            path = p+str(i)+".html"
            with open(path, "r") as f:
                contents = f.read()
                page = BeautifulSoup(contents, 'lxml')
            if page.find("table", id="disambigbox")!=None or page.find("table", id="setindexbox")!=None or page.find("table", id="noarticletext")!=None:
                row=[i]+['NA' for i in range(14)]
                tsv_writer = csv.writer(out_file, delimiter='\t')
                tsv_writer.writerow(row)
                #print('missing page',i)
                missing_pages.append(i)
            else:
                row = [i, get_title(page), get_intro(page), get_plot(page)]+ extract_infobox2(page)   
                tsv_writer = csv.writer(out_file, delimiter='\t')
                tsv_writer.writerow(row)
    print(missing_pages)

In [0]:
movies_1= pd.read_csv("/content/drive/My Drive/wiki_dataset/tsv_files/Movies1_tsv.tsv",sep='\t')
movies_2= pd.read_csv("/content/drive/My Drive/wiki_dataset/tsv_files/Movies2_tsv.tsv",sep='\t')
movies_3= pd.read_csv("/content/drive/My Drive/wiki_dataset/tsv_files/Movies3_tsv.tsv",sep='\t')